In [ ]:
import warnings

warnings.filterwarnings("ignore")

In [ ]:
from pathlib import Path

data_folder = Path("../../data")

prod_data_folder: Path = data_folder.joinpath("30_prod/")
model_artifact = Path("../../models/artifacts")

In [ ]:
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import pandas as pd

plt.rcParams.update({"figure.figsize": (8, 6), "text.usetex": True})

GENERATOR_KEYWORD: str = "GENERATOR"
DISCRIMINATOR_KEYWORD: str = "DISCRIMINATOR"

MODEL_TYPE: str = "GAN_CNN"

AAPL_TICKER: str = "AAPL"
ACWI_TICKER: str = "ACWI"
JPM_TICKER: str = "JPM"

aapl_stock_filename: str = f"{AAPL_TICKER}.csv"
acwi_stock_filename: str = f"{ACWI_TICKER}.csv"
jpm_stock_filename: str = f"{JPM_TICKER}.csv"

aapl_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(aapl_stock_filename))
acwi_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(acwi_stock_filename))
jpm_df: pd.DataFrame = pd.read_csv(prod_data_folder.joinpath(jpm_stock_filename))

aapl_df["Date"] = pd.to_datetime(aapl_df["Date"], format="%Y-%m-%d")
acwi_df["Date"] = pd.to_datetime(acwi_df["Date"], format="%Y-%m-%d")
jpm_df["Date"] = pd.to_datetime(jpm_df["Date"], format="%Y-%m-%d")

aapl_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
acwi_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
jpm_min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

aapl_min_max_scaler.fit(aapl_df[["Close"]])
acwi_min_max_scaler.fit(acwi_df[["Close"]])
jpm_min_max_scaler.fit(jpm_df[["Close"]])

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim, seq_length=60, n_features=1):
        super(Generator, self).__init__()
        self.seq_length = seq_length
        self.n_features = n_features
        self.init_length = seq_length // 4
        self.init_channels = 128

        self.fc = nn.Sequential(
            nn.Linear(latent_dim, self.init_channels * self.init_length),
            nn.BatchNorm1d(self.init_channels * self.init_length),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.deconv = nn.Sequential(
            nn.ConvTranspose1d(
                self.init_channels, 64, kernel_size=4, stride=2, padding=1
            ),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.2, inplace=True),
            nn.ConvTranspose1d(64, n_features, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.fc(z)
        out = out.view(z.size(0), self.init_channels, self.init_length)
        out = self.deconv(out)
        out = out.permute(0, 2, 1)
        return out


class Discriminator(nn.Module):
    def __init__(self, seq_length=60, n_features=1):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Conv1d(n_features, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            nn.Conv1d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
            nn.Conv1d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Dropout(0.25),
        )
        self.fc = nn.Sequential(nn.Flatten(), nn.Linear(256 * 7, 1), nn.Sigmoid())

    def forward(self, x):
        x = x.permute(0, 2, 1)
        features = self.model(x)
        validity = self.fc(features)
        return validity

### Apple

In [ ]:
aapl_generator_model_filename: str = (
    f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{AAPL_TICKER}.pth"
)
aapl_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{AAPL_TICKER}.pth"
)

aapl_generator = Generator(100)
aapl_discriminator = Discriminator()

aapl_generator.load_state_dict(
    torch.load(model_artifact / aapl_generator_model_filename)
)
aapl_discriminator.load_state_dict(
    torch.load(model_artifact / aapl_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

aapl_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(1, 1, 100)
    for i in range(1):
        output_tensor = aapl_generator(input_tensor[i])
        ax.plot(
            aapl_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("AAPL CNN Generator Model Architecture")
plt.show()

### ACWI

In [ ]:
acwi_generator_model_filename: str = (
    f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{ACWI_TICKER}.pth"
)
acwi_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{ACWI_TICKER}.pth"
)

acwi_generator = Generator(100)
acwi_discriminator = Discriminator()

acwi_generator.load_state_dict(
    torch.load(model_artifact / acwi_generator_model_filename)
)
acwi_discriminator.load_state_dict(
    torch.load(model_artifact / acwi_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

acwi_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(1, 1, 100)
    for i in range(1):
        output_tensor = acwi_generator(input_tensor[i])
        ax.plot(
            acwi_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("ACWI CNN Generator Model Architecture")
plt.show()

### JPM

In [ ]:
jpm_generator_model_filename: str = f"{GENERATOR_KEYWORD}_{MODEL_TYPE}_{JPM_TICKER}.pth"
jpm_discriminator_model_filename: str = (
    f"{DISCRIMINATOR_KEYWORD}_{MODEL_TYPE}_{JPM_TICKER}.pth"
)

jpm_generator = Generator(100)
jpm_discriminator = Discriminator()

jpm_generator.load_state_dict(torch.load(model_artifact / jpm_generator_model_filename))
jpm_discriminator.load_state_dict(
    torch.load(model_artifact / jpm_discriminator_model_filename)
)

In [ ]:
fig = plt.figure(figsize=(8, 6))
ax = fig.add_subplot(111)

jpm_generator.eval()

with torch.no_grad():
    input_tensor = torch.randn(4, 1, 100)
    for i in range(4):
        output_tensor = jpm_generator(input_tensor[i])
        ax.plot(
            jpm_min_max_scaler.inverse_transform(output_tensor[0].numpy()),
            label=f"Sample {i}",
        )

ax.set_title("JPM CNN Generator Model Architecture")
plt.show()